In [32]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Use the same synthetic data as above
np.random.seed(102)
num_points = 340
body_mass_g = np.random.uniform(2700, 6300, num_points)
m_true = 0.015
c_true = 120
noise = np.random.normal(0, 5, num_points)
flipper_length_mm = m_true * body_mass_g + c_true + noise

df = pd.DataFrame({
    'body_mass_g': body_mass_g,
    'flipper_length_mm': flipper_length_mm
})

df['body_mass_std'] = (df['body_mass_g'] - df['body_mass_g'].mean()) / df['body_mass_g'].std()
df['flipper_length_std'] = (df['flipper_length_mm'] - df['flipper_length_mm'].mean()) / df['flipper_length_mm'].std()
X = df[['body_mass_std', 'flipper_length_std']].values
X[:, 0] *= 2  # Scale x-axis for visual effect

count_show = 20

# Pick fixed indices for reproducibility (20, not 5)
rng = np.random.default_rng(51)
indices = rng.choice(len(X), replace=False, size=count_show)
X_subset = X[indices]

# Compute optimal SVD/PCA direction
U, S, Vt = np.linalg.svd(X, full_matrices=False)
opt_direction = Vt[0]
opt_direction = opt_direction / np.linalg.norm(opt_direction)
opt_theta = np.arctan2(opt_direction[1], opt_direction[0])

def wrap_angle(theta):
    return (theta + 2 * np.pi) % (2 * np.pi)

def make_line_pts(theta, vmin, vmax, extend=0.2):
    direction = np.array([np.cos(theta), np.sin(theta)])
    range_min = vmin - extend * (vmax - vmin)
    range_max = vmax + extend * (vmax - vmin)
    xs = [range_min * direction[0], range_max * direction[0]]
    ys = [range_min * direction[1], range_max * direction[1]]
    return np.array([xs, ys]).T

def get_projections(X_pts, theta):
    direction = np.array([np.cos(theta), np.sin(theta)])
    direction = direction / np.linalg.norm(direction)
    ts = X_pts @ direction
    proj_points = np.outer(ts, direction)
    return proj_points

vals_for_range = np.concatenate([X[:, 0], X[:, 1]])
vabs = np.max(np.abs(vals_for_range))
vmin, vmax = -vabs, vabs

from plotly.subplots import make_subplots

n_steps = 181
thetas = np.linspace(-np.pi, np.pi, n_steps) + opt_theta

# Hide ticks/labels completely under the slider (just the clean slider)
theta_labels = [''] * n_steps
start_ix = n_steps // 2

xlim = (-3, 2)
ylim = (-2, 1.5)
xticks = [-3, -2, -1, 0, 1, 2]
xticktext = ['', '', '', '', '', '']
yticks = [-2, -1, 0, 1, 1.5]
yticktext = ['', '', '', '', '']

def make_frame(theta):
    direction = np.array([np.cos(theta), np.sin(theta)])
    proj_points = get_projections(X_subset, theta)
    line_seg = make_line_pts(theta, vmin, vmax)
    traces = []
    traces.append(go.Scatter(
        x=X[:, 0], y=X[:, 1],
        mode='markers',
        marker=dict(color="#3D81F6", size=8, opacity=0.25),
        showlegend=False,
        hoverinfo='skip'
    ))
    traces.append(go.Scatter(
        x=X_subset[:, 0], y=X_subset[:, 1],
        mode='markers',
        marker=dict(color="#3D81F6", size=13),
        name="Subset",
        showlegend=False
    ))
    traces.append(go.Scatter(
        x=proj_points[:, 0], y=proj_points[:, 1],
        mode='markers',
        marker=dict(color="orange", size=13),
        name="Projections",
        showlegend=False
    ))
    traces.append(go.Scatter(
        x=line_seg[:, 0], y=line_seg[:, 1],
        mode='lines',
        line=dict(color="orange", width=5, dash='solid'),
        opacity=0.5,
        name="Direction"
    ))
    for i in range(count_show):
        traces.append(go.Scatter(
            x=[X_subset[i, 0], proj_points[i, 0]],
            y=[X_subset[i, 1], proj_points[i, 1]],
            mode='lines',
            line=dict(color="#d81a60", width=2, dash='dot'),
            showlegend=False,
            hoverinfo='skip'
        ))
    return traces

frames = []
for ti, theta in enumerate(thetas):
    traces = make_frame(theta)
    frame = go.Frame(
        data=traces,
        name=str(ti),
        traces=list(range(len(traces)))
    )
    frames.append(frame)

init_traces = make_frame(thetas[start_ix])
fig = go.Figure(data=init_traces, frames=frames)

# Hide any number-line marks/ticks under the slider
steps = []
for i, theta in enumerate(thetas):
    step = dict(
        method="animate",
        args=[
            [str(i)],
            {"mode": "immediate", "frame": {"duration": 0, "redraw": True}, "transition": {"duration": 0}}
        ],
        label=''  # No tick labels at all
    )
    steps.append(step)

sliders = [dict(
    steps=steps,
    active=start_ix,
    transition={'duration': 0},
    x=0.2, y=0.01,
    currentvalue=dict(
        visible=False,
        xanchor='center'
    ),
    len=0.75,
    pad={"b": 20, "t": 20},   # Minimize slider padding for clean look
    ticklen=0,              # Hide slider ticks completely
    tickwidth=0,            # Hide tick lines
    tickcolor="rgba(0,0,0,0)"  # Make ticks transparent
)]

fig.update_layout(
    sliders=sliders,
    showlegend=False,
    plot_bgcolor='#faf9f5',  # set background
    paper_bgcolor='#faf9f5',
    margin=dict(l=30, r=30, t=20, b=0),
    font=dict(
        family="Avenir, system-ui, Helvetica, Arial, sans-serif",
        color="black"
    ),
    xaxis_range=list(xlim),
    yaxis_range=list(ylim),
    xaxis=dict(
        tickvals=xticks,
        ticktext=xticktext,
        gridcolor='#f0f0f0',
        showline=True,
        linecolor="black",
        linewidth=1,
        zeroline=False,
        # zerolinewidth=3,
        # zerolinecolor="black"
    ),
    yaxis=dict(
        tickvals=yticks,
        ticktext=yticktext,
        gridcolor='#f0f0f0',
        showline=True,
        linecolor="black",
        linewidth=1,
        zeroline=False,
        # zerolinewidth=3,
        # zerolinecolor="black"
    ),
    title=None,
    width=400,
    height=350
)

fig.show()

fig.write_html(
    "pca_orthogonal_errors.html",
    config=dict(
        displayModeBar=False,
        displaylogo=False,
        modeBarButtonsToRemove=[
            'sendDataToCloud',
            'zoomIn2d',
            'zoomOut2d',
            'autoScale2d',
            'resetScale2d',
            'toImage',
            'select2d',
            'lasso2d',
            'toggleSpikelines'
        ]
    ),
    auto_play=False
)
